In [6]:
import torch

model = torch.load("exported_models/backbone_1_1_1_1.pt")
model

skin_backbone(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): BBoneELAN(
      (layers): Sequential(
        (0): 

In [7]:
import os
from typing import Dict

import onnx
import tensorflow as tf
import torch
from onnx_tf.backend import prepare

model = torch.load("exported_models/backbone_1_1_1_1.pt")

def arch2tflite(name: str, out_dir: str):

    base_path = os.path.join(out_dir, name)
    pytorch_model_path = base_path + ".pt"
    onnx_model_path = base_path + ".onnx"
    tf_model_path = base_path + ".pb"
    tflite_model_path = base_path + ".tflite"

    pytorch_model = torch.load(pytorch_model_path)

    # model to onnx
    if 'backbone' in name:
        dummy_input = torch.randn(1, 3, 640, 640)
    else:
        dummy_input = torch.randn(1, 1024, 20, 20)
    torch.onnx.export(pytorch_model, dummy_input, onnx_model_path, input_names=["input"])

    # model to tf
    onnx_model = onnx.load(onnx_model_path)
    tf_rep = prepare(onnx_model)
    tf_rep.export_graph(tf_model_path)

    # model to tflite
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
    tflite_model = converter.convert()
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)

arch2tflite('backbone_1_1_1_1', 'exported_models')

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
from nas.supernet.supernet_yolov7 import YOLOSuperNet
from latency_predictor.arch_utils import *
import time
from os.path import join
from tqdm.auto import tqdm

device = 'cpu'
supernet = YOLOSuperNet(cfg='./yaml/yolov7_dynamicsupernet.yml').to(device)
# supernet.train()

model_out_dir = './exported_models'

backbone_lut = {}
head_lut = {}

BACKBONE_MAX_DEPTH = 2 # should be 6
HEAD_MAX_DEPTH = 2 # should be 8

# cal. latency for backbone
for i in range(1, BACKBONE_MAX_DEPTH):
    for j in range(1, BACKBONE_MAX_DEPTH):
        for k in range(1, BACKBONE_MAX_DEPTH):
            for m in range(1, BACKBONE_MAX_DEPTH):
                supernet.set_active_subnet([i,j,k,m,1,1,1,1]) # Maximum: [5,5,5,5,7,7,7,7]
                subnet = supernet.get_active_subnet()

                b_arch, _, save = split_backbone_head(subnet)

                backbone = skin_backbone(b_arch, save)
                torch.save(backbone, join(model_out_dir, f'backbone_{i}_{j}_{k}_{m}.pt'))
#                 #########이부분이 galaxy 측정으로 대체되어야 함
#                 backbone = skin_backbone(b_arch, save)
#                 img = torch.rand(1, 3, 640, 640).to(device)
#                 s = time.time()
#                 y = backbone(img)
#                 latency = (time.time()-s )*1000 # ms :: 실제로 측정할때는 1000 번이상 측정해서 평균값으로

#                 backbone_lut[str(i)+str(j)+str(k)+str(m)] = latency

# print("backbone completed")

# cal. latency for head
for i in range(1, HEAD_MAX_DEPTH):
    for j in range(1, HEAD_MAX_DEPTH):
        for k in range(1, HEAD_MAX_DEPTH):
            for m in range(1, HEAD_MAX_DEPTH):
                supernet.set_active_subnet([1,1,1,1,i,j,k,m]) # Maximum: [5,5,5,5,7,7,7,7]
                subnet = supernet.get_active_subnet()

                _, h_arch, save = split_backbone_head(subnet)

                #########이부분이 galaxy 측정으로 대체되어야 함
                # head = skin_head(h_arch, save)
                # img = torch.rand(1, 1024, 20, 20).to(device)
                # s = time.time()
                # y = head(img)
                # latency = (time.time()-s )*1000 # ms :: 실제로 측정할때는 1000 번이상 측정해서 평균값으로

                # head_lut[str(i)+str(j)+str(k)+str(m)] = latency




KeyboardInterrupt: 

In [2]:
import json

file_path = "latency_predictor/lut/head.json"

with open(file_path, 'w') as f:
    json.dump(head_lut, f)